This is a program that will train a model to identify and assign hits to tracks.
Written by Daniel Zurawski & Keshav Kapoor for Fermilab Summer 2017 internship.

In [10]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import keras

The below code is borrowed from a DS&HEP tutorial.
It is used to graph the histories after fitting a model.

In [2]:
def show_losses( histories ):
    plt.figure(figsize=(10,10))
    #plt.ylim(bottom=0)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Error by Epoch')
    colors=[]
    do_acc=False
    for label,loss in histories:
        color = tuple([0.1, 0.1, 0.1])
        colors.append(color)
        l = label
        vl= label+" validation"
        if 'acc' in loss.history:
            l+=' (acc %2.4f)'% (loss.history['acc'][-1])
            do_acc = True
        if 'val_acc' in loss.history:
            vl+=' (acc %2.4f)'% (loss.history['val_acc'][-1])
            do_acc = True
        plt.plot(loss.history['loss'], label=l, color=color)
        if 'val_loss' in loss.history:
            plt.plot(loss.history['val_loss'], lw=2, ls='dashed', label=vl, color=color)


    plt.legend()
    plt.yscale('log')
    plt.show()
    if not do_acc: return
    plt.figure(figsize=(10,10))
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    for i,(label,loss) in enumerate(histories):
        color = tuple([0.0, 0.0, 1.0])
        if 'acc' in loss.history:
            plt.plot(loss.history['acc'], lw=2, label=label+" accuracy", color=color)
        if 'val_acc' in loss.history:
            plt.plot(loss.history['val_acc'], lw=2, ls='dashed', label=label+" validation accuracy", color=color)
    plt.legend(loc='lower right')
    plt.show()

Here, we define a LinearTracker class.
This class is used to load input and output from a .csv file in the correct format for training the model.

In [3]:
class LinearTracker():
    """ An object that classifies particles to tracks after an event. """    
    def __init__(self, dataframe, model=None):
        """ Initialize a LinearTracker.
            @param dataframe - pd.DataFrame - used to pick tracks from.
                The headers should contain: ("id", "z", "r", "phi").
            @param model - keras model - A network model that the tracker will
                use to classify particles.
            @return Nothing
        """
        self.model     = model     # keras model to figure out tracks.
        self.dataframe = dataframe # pandas.DataFrame for picking tracks.
        self.input     = None      # input to train model on.
        self.output    = None      # output to train model on.
    # END function __init__
    
    def load_data(self, num_events, tracks_per_event, track_size, noise_per_event):
        """ Load input and output data from this object's dataframe.
            @param num_events - int - The number of events to generate.
            @param tracks_per_event - int - The number of tracks per event.
            @param track_size - int - The number of hits per track.
            @param noise_per_event - int - The number of hits with no track.
            @return Nothing
                However, self.input and self.output become numpy arrays.
                self.input is collection of hits of shape:
                    (num_events, hits_per_event, 3)
                self.output is list of probability matrices of shape:
                    (num_events, hits_per_event, tracks_per_event)
        """
        hits_per_event = (track_size * tracks_per_event) + noise_per_event
        labels = ["phi", "r", "z"]
        groups = self.dataframe[["id", "r", "phi", "z"]].groupby("id")
        goods  = groups.filter(lambda track: len(track) == track_size)
        bads   = groups.filter(lambda track: len(track) != track_size)
        
        # Populate input and output with data.
        goods_group = [g[1] for g in list(goods.groupby("id"))]
        self.input  = np.zeros((num_events, hits_per_event, len(labels)))
        self.output = np.zeros((num_events, hits_per_event, tracks_per_event))
        for n in range(num_events):
            # Retrieve a sample of tracks.
            tracks = random.sample(goods_group, tracks_per_event)
            
            # Make a mapping from track ID to index within a matrix.
            T2I = self.__get_matrix_map__(tracks) # Track to Index
            
            # Make some noise hits to add.
            noise  = bads.sample(noise_per_event)
            hits   = pd.concat(tracks + [noise]).sort_values(labels)
            
            self.__populate_input__(hits, labels, n)
            self.__populate_output__(hits, T2I, n)
    # END FUNCTION load_data
   ''' 
    def __data_converter__(data):
        phi, r, z = data
        return (np.cos(phi)*r, np.sin(phi)*r, z)
    
    def __track_colors__(eventID):
        data = self.output[eventID]
        colors = ['0426c1', 'c10404', '09A016', '9709A0', 'ff5733', '0x0000']
        nonZeros = np.nonzero(data)
        hitColors = np.zeros((data[1].size,6))
        
        for i, hit in enumerate(nonZeros):
            hitColors[i, 0] = colors[hit[]]
    
    def create_plot(eventID):
        hits = np.array([self.__data_converter__(x) for x in self.input[eventID]])
        frame = pd.DataFrame(data=hits)
        frame.groupby()
    '''
    def __populate_input__(self, hits, labels, event_index):
        self.input[event_index, :] = hits[labels].values
    # END FUNCTION __populate_input__
    
    def __populate_output__(self, hits, mapping, event_index):
        for t, track_ID in enumerate(hits["id"]):
            index = mapping.get(track_ID)
            if index is not None:
                self.output[event_index, t, index] = 1
    # END FUNCTION __populate_output__
        
    def __get_matrix_map__(self, tracks):
        L = pd.concat([T.sort_values(["r"]).head(1) for T in tracks])
        L.sort_values(["phi", "z"], inplace=True)
        return dict((hit, idx) for idx, hit in enumerate(L["id"]))
    # END FUNCTION __get_matrix_map__
# END CLASS LinearTracker

Below is how to create a LinearTracker and how to load data into it. It is important to note that after construction, a LinearTracker must call its load_data() function with user specifications for how data should be loaded.

If you get a ValueError describing how the population is not large enough for the sample, then that means that the data
loaded in from the .csv file does not contain enough tracks of size 'track_size'. Try to either load in a larger
population or change the 'track_size' variable to a different positive integer.

In [4]:
filename  = ('../Data Sets/linear_data_5k.csv')
dataframe = pd.read_csv(filename)
tracker   = LinearTracker(dataframe)

Load the data into the input and output member variables of LinearTracker.

In [5]:
np.random.seed(7)
tracker.load_data(num_events=512, tracks_per_event=5, track_size=4, noise_per_event=5)
print("Ding! All done.")

Ding! All done.


Let's take a look at the input and output training data.

In [6]:
from IPython.display import display,HTML

def multi_column_df_display(list_dfs, cols=2):
    """ Code by David Medenjak responding to StackOverflow question found here:
        https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
        Displays a list of dataframes in IPython as a table with cols number of columns.
    """
    html_table = "<table style='width:100%; border:0px'>{content}</table>"
    html_row = "<tr style='border:0px'>{content}</tr>"
    html_cell = "<td style='width:{width}%;vertical-align:top;border:0px'>{{content}}</td>"
    html_cell = html_cell.format(width=100/cols)

    cells = [ html_cell.format(content=df.to_html()) for df in list_dfs ]
    cells += (cols - (len(list_dfs)%cols)) * [html_cell.format(content="")] # pad
    rows = [ html_row.format(content="".join(cells[i:i+cols])) for i in range(0,len(cells),cols)]
    display(HTML(html_table.format(content="".join(rows))))
# END FUNCTION multi_column_df_display

input_cols  = ["phi", "r", "z"]
output_cols = ["T{}".format(i) for i in range(tracker.output.shape[2])]
show_max    = 2

if show_max is not None and show_max > 0 and show_max < len(tracker.input):
    print("Displaying the first {} inputs and outputs.".format(show_max))
    input_frames  = [pd.DataFrame(data=tracker.input[i], columns=input_cols) for i in range(show_max)]
    output_frames = [pd.DataFrame(data=tracker.output[i].astype(int), columns=output_cols) for i in range(show_max)]
else:
    print("Displaying all of input and output.")
    input_frames  = [pd.DataFrame(data=matrix, columns=input_cols)  for matrix in tracker.input]
    output_frames = [pd.DataFrame(data=matrix.astype(int), columns=output_cols) for matrix in tracker.output]
    
df_list  = []
for i in range(len(input_frames)):    
    df_list.append(input_frames[i])
    df_list.append(output_frames[i])

print("Input shape:  {}".format(tracker.input.shape))
print("Output shape: {}".format(tracker.output.shape))
multi_column_df_display(df_list)

Displaying the first 2 inputs and outputs.
Input shape:  (512, 25, 3)
Output shape: (512, 25, 5)


,phi,r,z
0,0.539617,40.0,2.827315
1,1.034402,100.0,-37.702537
2,4.471363,40.0,28.020044
3,4.471363,70.0,28.020044
4,4.471363,100.0,28.020044
5,4.471363,130.0,28.020044
6,4.799925,40.0,8.562432
7,5.230007,40.0,-5.142094
8,5.365522,40.0,31.074293
9,5.849170,40.0,-49.032144


Let's now try to load a model into our tracker.

In [7]:
from keras.layers import Dense, Activation, LSTM, Dropout, Flatten
from keras.models import Sequential
from keras.optimizers import SGD, RMSprop
from keras.layers.wrappers import TimeDistributed

In [8]:
input_shape = tracker.input[0].shape # Shape of an event.
output_shape = len(tracker.output[0][0]) # Number of tracks per event
print("Input Shape:  {}".format(input_shape))
print("Output Shape: {}".format(output_shape))

kinit = 'uniform' # kernel_initializer
act   = 'relu' # activation
opt   = 'rmsprop' # optimizer

tracker.model = Sequential()
tracker.model.add(LSTM(64, return_sequences=True, input_shape=input_shape, dropout=.2, recurrent_dropout=0.2))
tracker.model.add(LSTM(64, return_sequences=True, dropout=.2, recurrent_dropout=.2))
tracker.model.add(LSTM(64, return_sequences=True, dropout=.2, recurrent_dropout=.2))
tracker.model.add(Dense(output_shape, activation='softmax'))

tracker.model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
#tracker.model.summary()

Input Shape:  (25, 3)
Output Shape: 5


In [9]:
batch_size = 32
hist = tracker.model.fit(tracker.input, tracker.output, epochs=128, batch_size=batch_size,
                         verbose=0, validation_split=0.25,
                         callbacks=[keras.callbacks.ModelCheckpoint(filepath='simple.h5', verbose=0)])
print("Ding! All done.")

Ding! All done.


In [11]:
print(tracker.model.predict(tracker.input))

[[[  9.96190548e-01   1.11710595e-03   2.28421166e-04   6.73626608e-04
     1.79033028e-03]
  [  9.99970436e-01   1.58785733e-05   1.09007630e-07   6.62841842e-07
     1.29071323e-05]
  [  9.99956250e-01   3.82454564e-05   3.89812769e-08   2.29730560e-07
     5.26928534e-06]
  ..., 
  [  1.69474806e-04   1.88901129e-06   7.38958124e-06   1.15579879e-03
     9.98665452e-01]
  [  2.19657770e-04   2.70769324e-05   2.92422083e-05   3.30325426e-03
     9.96420741e-01]
  [  2.04296535e-04   1.34433765e-04   1.33437221e-04   6.00951072e-03
     9.93518293e-01]]

 [[  9.96328413e-01   1.10983371e-03   2.16205022e-04   6.24824665e-04
     1.72070484e-03]
  [  9.99969721e-01   1.69863142e-05   1.08445967e-07   6.97310099e-07
     1.24705139e-05]
  [  9.99974251e-01   2.40858189e-05   4.53036009e-09   8.52637161e-09
     1.64706773e-06]
  ..., 
  [  1.33935449e-04   1.23983564e-05   1.04834044e-05   4.94173670e-04
     9.99349058e-01]
  [  1.79622584e-04   1.00426158e-04   8.18906701e-05   1.5947

It's time to graph the history of the neural network.

In [ ]:
score, acc = tracker.model.evaluate(tracker.input, tracker.output, batch_size=batch_size)
print('\nTest score:', score)
print('Test accuracy:', acc)
show_losses([("categorical cross entropy", hist)])